In [3]:
import sys
import re 
import pandas 
print(pandas.__file__)
import numpy as np 
from glob import glob
import IPython
from tqdm import tqdm
import pickle
from keras.utils import np_utils
import play

C:\Users\adity\Anaconda3\lib\site-packages\pandas\__init__.py


Using TensorFlow backend.


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
from music21 import converter, instrument, note, chord, stream

In [5]:
songs = glob('Jazz/*.mid')

In [6]:
songs = songs[:7]
print(songs)


['Jazz\\2ndMovementOfSinisterFootwear.mid', 'Jazz\\55Dive.mid', 'Jazz\\5To10.mid', 'Jazz\\634-5789.mid', 'Jazz\\914.mid', 'Jazz\\ABC.mid', 'Jazz\\ACertainSmile.mid']


In [7]:
def get_notes():
    notes = []
    for file in songs:
        # converting .mid file to stream object
        midi = converter.parse(file)
        notes_to_parse = []
        try:
            # Given a single stream, partition into a part for each unique instrument
            parts = instrument.partitionByInstrument(midi)
        except:
            pass
        if parts: # if parts has instrument parts 
            notes_to_parse = parts.parts[0].recurse()
        else:
            notes_to_parse = midi.flat.notes
    
        for element in notes_to_parse: 
            if isinstance(element, note.Note):
                # if element is a note, extract pitch
                notes.append(str(element.pitch))
            elif(isinstance(element, chord.Chord)):
                # if element is a chord, append the normal form of the 
                # chord (a list of integers) to the list of notes. 
                notes.append('.'.join(str(n) for n in element.normalOrder))
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    
    return notes

In [8]:
def prepare_sequences(notes, n_vocab): 
    sequence_length = 100

    # Extract the unique pitches in the list of notes.
    pitchnames = sorted(set(item for item in notes))

    # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i: i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    
    n_patterns = len(network_input)
    
    # reshape the input into a format comatible with LSTM layers 
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input
    network_input = network_input / float(n_vocab)
    
    # one hot encode the output vectors
    network_output = np_utils.to_categorical(network_output)
    
    return (network_input, network_output)

In [9]:
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout, Flatten
def create_network(network_in, n_vocab): 
    """Create the model architecture"""
    model = Sequential()
    model.add(LSTM(128, input_shape=network_in.shape[1:], return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    print(model.summary())
  

    return model


In [10]:
from keras.callbacks import ModelCheckpoint
def train(model, network_input, network_output, epochs): 
    """
    Train the neural network
    """
    # Create checkpoint to save the best model weights.
    filepath = 'weights.best.music3.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
    
    model.fit(network_input, network_output, epochs=epochs, batch_size=32, callbacks=[checkpoint])

In [11]:
def train_network():
    """
    Get notes
    Generates input and output sequences
    Creates a model 
    Trains the model for the given epochs
    """
    
    epochs = 20
    with open('data/notes', 'rb') as filepath:notes = pickle.load(filepath)

    print('Notes processed')
    
    n_vocab = len(set(notes))
    print('Vocab generated')
    
    network_in, network_out = prepare_sequences(notes, n_vocab)
    print('Input and Output processed')
    
    model = create_network(network_in, n_vocab)
    print('Model created')
    
    print('Training in progress')
    train(model, network_in, network_out, epochs)
    print('Training completed')
    return model

In [12]:
### Train the model 
train_network()

Notes processed
Vocab generated
Input and Output processed
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 128)          66560     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 128)          131584    
_________________________________________________________________
flatten_1 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               3277056   
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)      

1184/1184 [==============================] - ETA: 11s - loss: 3.95 - ETA: 9s - loss: 3.9938 - ETA: 9s - loss: 3.947 - ETA: 9s - loss: 3.937 - ETA: 8s - loss: 3.949 - ETA: 8s - loss: 3.916 - ETA: 8s - loss: 3.927 - ETA: 7s - loss: 3.919 - ETA: 7s - loss: 3.924 - ETA: 7s - loss: 3.946 - ETA: 7s - loss: 3.944 - ETA: 6s - loss: 3.964 - ETA: 6s - loss: 3.983 - ETA: 6s - loss: 3.988 - ETA: 6s - loss: 3.967 - ETA: 5s - loss: 3.972 - ETA: 5s - loss: 3.972 - ETA: 5s - loss: 3.979 - ETA: 4s - loss: 3.993 - ETA: 4s - loss: 3.998 - ETA: 4s - loss: 3.991 - ETA: 4s - loss: 3.986 - ETA: 3s - loss: 3.993 - ETA: 3s - loss: 3.989 - ETA: 3s - loss: 3.984 - ETA: 2s - loss: 3.980 - ETA: 2s - loss: 3.991 - ETA: 2s - loss: 4.009 - ETA: 2s - loss: 4.011 - ETA: 1s - loss: 4.028 - ETA: 1s - loss: 4.040 - ETA: 1s - loss: 4.046 - ETA: 1s - loss: 4.045 - ETA: 0s - loss: 4.045 - ETA: 0s - loss: 4.047 - ETA: 0s - loss: 4.047 - 10s 9ms/step - loss: 4.0444
Epoch 8/20
1184/1184 [==============================] - ETA: 1

1184/1184 [==============================] - ETA: 11s - loss: 2.80 - ETA: 9s - loss: 2.9382 - ETA: 8s - loss: 2.821 - ETA: 7s - loss: 2.863 - ETA: 7s - loss: 2.894 - ETA: 7s - loss: 2.899 - ETA: 7s - loss: 2.937 - ETA: 6s - loss: 2.961 - ETA: 6s - loss: 2.962 - ETA: 6s - loss: 3.004 - ETA: 6s - loss: 3.006 - ETA: 5s - loss: 3.003 - ETA: 5s - loss: 3.026 - ETA: 5s - loss: 3.046 - ETA: 5s - loss: 3.044 - ETA: 4s - loss: 3.034 - ETA: 4s - loss: 3.063 - ETA: 4s - loss: 3.079 - ETA: 4s - loss: 3.077 - ETA: 3s - loss: 3.083 - ETA: 3s - loss: 3.076 - ETA: 3s - loss: 3.075 - ETA: 3s - loss: 3.054 - ETA: 2s - loss: 3.066 - ETA: 2s - loss: 3.080 - ETA: 2s - loss: 3.089 - ETA: 2s - loss: 3.083 - ETA: 2s - loss: 3.083 - ETA: 1s - loss: 3.086 - ETA: 1s - loss: 3.087 - ETA: 1s - loss: 3.094 - ETA: 1s - loss: 3.110 - ETA: 0s - loss: 3.105 - ETA: 0s - loss: 3.109 - ETA: 0s - loss: 3.117 - ETA: 0s - loss: 3.119 - 8s 7ms/step - loss: 3.1153
Epoch 16/20
1184/1184 [==============================] - ETA: 1

In [13]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))
    
    print('Initiating music generation process.......')
    
    network_input = get_inputSequences(notes, pitchnames, n_vocab)
    normalized_input = network_input / float(n_vocab)
    model = create_network(normalized_input, n_vocab)
    print('Loading Model weights.....')
    model.load_weights('weights.best.music3.hdf5')
    print('Model Loaded')
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [14]:
def get_inputSequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
    
    network_input = np.reshape(network_input, (len(network_input), 100, 1))
    
    return (network_input)

In [15]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # Pick a random integer
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    # pick a random sequence from the input as a starting point for the prediction
    pattern = list(network_input[start])
    prediction_output = []
    
    print('Generating notes........')

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)
        
        # Predicted output is the argmax(P(h|D))
        index = np.argmax(prediction)
        # Mapping the predicted interger back to the corresponding note
        result = int_to_note[index]
        # Storing the predicted output
        prediction_output.append(result)

        pattern.append(index)
        # Next input to the model
        pattern = pattern[1:len(pattern)]

    print('Notes Generated...')
    return prediction_output

In [16]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    
    print('Saving Output file as midi....')

    midi_stream.write('midi', fp='test_output6.mid')

In [17]:
#### Generate a new jazz music 
generate()

Initiating music generation process.......
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 128)          66560     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 128)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 128)          131584    
_________________________________________________________________
flatten_2 (Flatten)          (None, 12800)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               3277056   
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (De

PermissionError: [Errno 13] Permission denied: 'test_output6.mid'

In [ ]:
### Play the Jazz music
play.play_midi('test_output6.mid')